<a href="https://colab.research.google.com/github/Carolinsrainbow/UC_AnalisisdeDatosEnPython/blob/main/TF_Cajeros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Utilice el archivo DatosQF78.xlsx, en el que hay dos hojas de información, y las columnas que indican los días de la semana o los turnos corresponden a días o turnos en que se pueden reabastecer los cajeros automáticos. Realice sus cálculos, desarrollos y análisis en Python para responder las siguientes preguntas.

In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

In [4]:
# Cargamos los datos del excel
df = pd.read_excel("f1_DatosQF78.xlsx", )
df.head()

,Cajero,Coordenada en X,Coordenada en Y,Costo Fijo por Stock Out,Costo Variable por Stock Out,Duración de la recarga [Horas],Lunes,Martes,Miércoles,Jueves,Viernes,Sábado,Domingo,Turno mañana,Turno tarde,Turno noche
0,Cajero 1,50,104,0.39,0.04,0.2,1,1,1,1,1,1,1,0,0,1
1,Cajero 2,24,24,0.37,0.02,0.2,0,1,1,1,1,1,0,1,1,1
2,Cajero 3,9,48,0.44,0.05,0.3,1,1,1,1,0,0,1,1,1,1
3,Cajero 4,79,73,0.30,0.05,0.2,1,1,0,1,1,0,1,1,1,1
4,Cajero 5,13,20,0.23,0.04,0.3,1,1,0,1,1,1,1,1,1,1


Pregunta 1: ¿Cuántos cajeros pueden ser reabastecidos al menos 4 días a la semana?


In [7]:
# Calcular el número de días de reabastecimiento para cada cajero
df['Días Reabastecimiento'] = df[['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']].sum(axis=1)

# Filtrar los cajeros que pueden ser reabastecidos al menos 4 días a la semana
atms_restocked_4_days = df[df['Días Reabastecimiento'] >= 4]

# Contar el número de estos cajeros
number_of_atms = atms_restocked_4_days.shape[0]

number_of_atms

212

Pregunta 2: ¿Cuántos cajeros puedes ser reabastecidos al menos 4 días a la semana, que al menos uno de estos días sea en fin de semana y que no tenga turno de noche?

In [9]:
# Calcular el número de días de reabastecimiento para cada cajero
df['Días Reabastecimiento'] = df[['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']].sum(axis=1)

# Filtrar los cajeros que pueden ser reabastecidos al menos 4 días a la semana
# incluyendo al menos un día durante el fin de semana y sin turno de noche
atms_restocked_4_days_weekend_no_night = df[(df['Días Reabastecimiento'] >= 4) &
                                            ((df['Sábado'] == 1) | (df['Domingo'] == 1)) &
                                            (df['Turno noche'] == 0)]

# Contar el número de estos cajeros
number_of_atms_weekend_no_night = atms_restocked_4_days_weekend_no_night.shape[0]
number_of_atms_weekend_no_night


53

Pregunta 3: ¿Cuánto es el total de dinero (Monto) que se retiró del cajero 214?

Escribe el número con dos decimales, puedes separarlos con coma o punto.

In [13]:
 #Cargamos la sheet Retiros
 retiros_data = pd.read_excel('f1_DatosQF78.xlsx', sheet_name='Retiros')
 retiros_data.head()

,Día,Cajero,Monto,Hora de la transacción
0,1,Cajero 196,2.70,0:0
1,1,Cajero 14,2.92,0:1
2,1,Cajero 143,5.69,0:3
3,1,Cajero 142,6.74,0:4
4,1,Cajero 214,2.40,0:6


In [16]:
total_amount_atm_214 = retiros_data[retiros_data['Cajero'] == 'Cajero 214']['Monto'].sum()
total_amount_atm_214 = round(total_amount_atm_214, 2)
total_amount_atm_214

2072.49

Pregunta 4: Un cajero se define como rentable si el total de dinero (Monto) que se retiró en los 102 días registrados es mayor que 2200. Bajo esta definición, ¿cuántos de los cajeros son rentables?

In [25]:
# Asegurarse de que la columna 'Monto' es de tipo numérico
retiros_data['Monto'] = pd.to_numeric(retiros_data['Monto'], errors='coerce')

# Agrupar por 'Cajero' y contar el número de días únicos para cada cajero
days_count_per_atm = retiros_data.groupby('Cajero')['Día'].nunique()

# Filtrar los cajeros que tienen datos para exactamente 102 días
atms_with_102_days = days_count_per_atm[days_count_per_atm == 102].index

# Filtrar los datos originales para incluir solo estos cajeros
filtered_retiros_data = retiros_data[retiros_data['Cajero'].isin(atms_with_102_days)]

# Calcular el monto total retirado para cada cajero en los datos filtrados
total_withdrawals_per_atm = filtered_retiros_data.groupby('Cajero')['Monto'].sum()

# Filtrar los cajeros que tienen un monto total retirado mayor a 2200
rentable_atms = total_withdrawals_per_atm[total_withdrawals_per_atm > 2200]

# Contar el número de cajeros rentables
number_of_rentable_atms = rentable_atms.shape[0]

# Mostrar el resultado
print(f'Número de cajeros rentables: {number_of_rentable_atms}')

Número de cajeros rentables: 33


Pregunta 5
Al mirar el registro de retiros en cajeros, ¿cuántos retiros se hicieron entre las 2 y las 2:59 de la madrugada?

In [22]:
 #Convertir 'Hora de la transacción' a formato datetime para manejar el filtrado por hora
retiros_data['Hora de la transacción'] = pd.to_datetime(retiros_data['Hora de la transacción'], format='%H:%M')

# Filtrar las transacciones entre las 2:00 y las 2:59 AM
retiros_2_am = retiros_data[(retiros_data['Hora de la transacción'].dt.hour == 2)]

# Contar el número de dichas transacciones
number_of_retiros_2_am = retiros_2_am.shape[0]
number_of_retiros_2_am

4120

Considerando todos los retiros en los cajeros, ¿cuál de las siguientes alternativas indica la hora en la que hubo más y en la que hubo menos transacciones de retiro?  
1 punto


In [26]:
# Convertir la columna 'Hora de la transacción' a formato datetime
retiros_data['Hora de la transacción'] = pd.to_datetime(retiros_data['Hora de la transacción'], format='%H:%M')

# Extraer la hora de la columna 'Hora de la transacción'
retiros_data['Hora'] = retiros_data['Hora de la transacción'].dt.hour

# Contar el número de transacciones para cada hora
transacciones_por_hora = retiros_data['Hora'].value_counts().sort_index()

# Determinar la hora con más y menos transacciones
hora_mas_transacciones = transacciones_por_hora.idxmax()
hora_menos_transacciones = transacciones_por_hora.idxmin()

# Mostrar el resultado
print(f'La hora con más transacciones es: {hora_mas_transacciones}')
print(f'La hora con menos transacciones es: {hora_menos_transacciones}')

La hora con más transacciones es: 13
La hora con menos transacciones es: 0
